In [2]:
import os
from oss2 import SizedFileAdapter, determine_part_size
from oss2.models import PartInfo
import oss2

def uploadFileOSS(fname,local_fpath):
    # 阿里云主账号AccessKey拥有所有API的访问权限，风险很高。强烈建议您创建并使用RAM账号进行API访问或日常运维，请登录 https://ram.console.aliyun.com 创建RAM账号。
    auth = oss2.Auth('LTAIj6Mqlb0Jbqo5', 'q8oDj6FNA8b6HuVeMMWCVjgB4XFAtW')
    # Endpoint以杭州为例，其它Region请按实际情况填写。
    bucket = oss2.Bucket(auth, 'http://oss-cn-hangzhou.aliyuncs.com', 'eyisheng-hangzhou')

    key = 'shared/'+fname
    filename = local_fpath

    total_size = os.path.getsize(filename)
    # determine_part_size方法用来确定分片大小。
    part_size = determine_part_size(total_size, preferred_size=100000 * 1024)

    # 初始化分片。
    # 如果需要在初始化分片时设置文件存储类型，请在init_multipart_upload中设置相关headers，参考如下。
    # headers = dict()
    # headers["x-oss-storage-class"] = "Standard"
    # upload_id = bucket.init_multipart_upload(key, headers=headers).upload_id
    upload_id = bucket.init_multipart_upload(key).upload_id
    parts = []

    # 逐个上传分片。
    with open(filename, 'rb') as fileobj:
        part_number = 1
        offset = 0
        while offset < total_size:
            print('upload',part_number)
            num_to_upload = min(part_size, total_size - offset)
            # SizedFileAdapter(fileobj, size)方法会生成一个新的文件对象，重新计算起始追加位置。
            result = bucket.upload_part(key, upload_id, part_number,
                                        SizedFileAdapter(fileobj, num_to_upload))
            parts.append(PartInfo(part_number, result.etag))

            offset += num_to_upload
            part_number += 1

    # 完成分片上传。
    # 如果需要在完成分片上传时设置文件访问权限ACL，请在complete_multipart_upload函数中设置相关headers，参考如下。
    # headers = dict()
    # headers["x-oss-object-acl"] = oss2.OBJECT_ACL_PRIVATE
    # bucket.complete_multipart_upload(key, upload_id, parts, headers=headers)
    bucket.complete_multipart_upload(key, upload_id, parts)

    # 验证分片上传。
    with open(filename, 'rb') as fileobj:
        assert bucket.get_object(key).read() == fileobj.read()
        
    return


def downloadFileOSS(fname,local_fpath):
    # 阿里云主账号AccessKey拥有所有API的访问权限，风险很高。强烈建议您创建并使用RAM账号进行API访问或日常运维，请登录RAM控制台创建RAM账号。
    auth = oss2.Auth('LTAIj6Mqlb0Jbqo5', 'q8oDj6FNA8b6HuVeMMWCVjgB4XFAtW')
    # Endpoint以杭州为例，其它Region请按实际情况填写。
    bucket = oss2.Bucket(auth, 'http://oss-cn-hangzhou.aliyuncs.com', 'eyisheng-hangzhou')

    # 下载OSS文件到本地文件。如果指定的本地文件存在会覆盖，不存在则新建。
    #  <yourLocalFile>由本地文件路径加文件名包括后缀组成，例如/users/local/myfile.txt。
    bucket.get_object_to_file(os.path.join('shared/',fname), local_fpath)
    return

def listFileOSS():
    # 阿里云主账号AccessKey拥有所有API的访问权限，风险很高。强烈建议您创建并使用RAM账号进行API访问或日常运维，请登录RAM控制台创建RAM账号。
    auth = oss2.Auth('LTAIj6Mqlb0Jbqo5', 'q8oDj6FNA8b6HuVeMMWCVjgB4XFAtW')
    # Endpoint以杭州为例，其它Region请按实际情况填写。
    bucket = oss2.Bucket(auth, 'http://oss-cn-hangzhou.aliyuncs.com', 'eyisheng-hangzhou')

    # 列举存储空间下所有文件。
    for obj in oss2.ObjectIterator(bucket,prefix='shared/'):
        print(obj.key)
        
    return

In [6]:
listFileOSS()

shared/
shared/ResNet_model.zip
shared/bi-temp.txt
shared/biodata.zip
shared/biodata1.zip
shared/dataset_new.rar
shared/embedding-file.csv
shared/metallic1-0.zip
shared/metallic1-1.zip
shared/metallic1-2.zip
shared/metallic1-3.zip
shared/model-1.zip
shared/model-2.zip
shared/recommend-1-all.csv
shared/recommend-1-update.csv
shared/recommend-1.csv
shared/tuples.txt


In [ ]:
uploadFileOSS(fname='model-2.zip',local_fpath=r'/root/research/jupyter/git-metallic-5-5/metallic/model-1.zip')

In [ ]:
downloadFileOSS(,'/root/.cache/torch/checkpoints/ig_resnext101_32x16-c6f796b0.pth')